In [2]:
import ast
from typing import List, Dict, Optional

class FunctionExtractor(ast.NodeVisitor):
    def __init__(self, source: str):
        self.source = source
        self.functions: List[Dict[str, str]] = []
        self.class_stack: List[str] = []

    def visit_ClassDef(self, node: ast.ClassDef):
        self.class_stack.append(node.name)
        self.generic_visit(node)
        self.class_stack.pop()

    def visit_FunctionDef(self, node: ast.FunctionDef):
        self._add_function(node)
        self.generic_visit(node)

    def visit_AsyncFunctionDef(self, node: ast.AsyncFunctionDef):
        self._add_function(node)
        self.generic_visit(node)

    def _add_function(self, node):
        class_name: Optional[str] = self.class_stack[-1] if self.class_stack else None

        qualified_name = (
            f"{class_name}.{node.name}" if class_name else node.name
        )

        self.functions.append({
            "func_name": node.name,
            "qualified_name": qualified_name,
            "class_name": class_name,
            "is_async": isinstance(node, ast.AsyncFunctionDef),
            "func_source": ast.get_source_segment(self.source, node),
        })


def get_functions_from_file(file_path: str) -> List[Dict[str, str]]:
    with open(file_path, "r") as f:
        source = f.read()

    tree = ast.parse(source)
    extractor = FunctionExtractor(source)
    extractor.visit(tree)

    return extractor.functions


In [3]:
test_code = """
import os

def a_simple_fn():
    return "Hello World"

def fn_of_fn():
    print(a_simple_fn())

def nested_fn():
    def fn1():
        def fn2():
            return None
        return fn2
    return fn1

async def async_top_level():
    return 42

class MyService:

    def __init__(self, name):
        self.name = name

    def public_method(self):
        return self._helper()

    def _helper(self):
        return "private"

    @staticmethod
    def utility(x, y):
        return x + y

    @classmethod
    def factory(cls, name):
        return cls(name)

    async def async_method(self):
        return self.name

def my_decorator(fn):
    def wrapper(*args, **kwargs):
        return fn(*args, **kwargs)
    return wrapper

@my_decorator
def decorated_function():
    return "decorated"

square = lambda x: x * x
"""

file_path = "test_module.py"
with open(file_path, "w") as f:
    f.write(test_code)

functions = get_functions_from_file(file_path)

for fn in functions:
    print(
        f"Function: {fn['qualified_name']}, async={fn['is_async']}\n"
        f"{fn['func_source']}\n"
    )


Function: a_simple_fn, async=False
def a_simple_fn():
    return "Hello World"

Function: fn_of_fn, async=False
def fn_of_fn():
    print(a_simple_fn())

Function: nested_fn, async=False
def nested_fn():
    def fn1():
        def fn2():
            return None
        return fn2
    return fn1

Function: fn1, async=False
def fn1():
        def fn2():
            return None
        return fn2

Function: fn2, async=False
def fn2():
            return None

Function: async_top_level, async=True
async def async_top_level():
    return 42

Function: MyService.__init__, async=False
def __init__(self, name):
        self.name = name

Function: MyService.public_method, async=False
def public_method(self):
        return self._helper()

Function: MyService._helper, async=False
def _helper(self):
        return "private"

Function: MyService.utility, async=False
def utility(x, y):
        return x + y

Function: MyService.factory, async=False
def factory(cls, name):
        return cls(name)

F